# Gram--Schmidt and the QR Decomposition

In the previous workbook, we saw the definition of an _orthogonal set_ of vectors. Indeed, the set $V = \{v_1,\dots, v_k\}$ is an _orthogonal set_ if $v_i^\top v_j = 0$ for all $i\neq j$. The set $V$ is _orthonormal_ if in addition to being orthogonal, we have that $\|v_i\| = 1$ for all $i=1,\dots,k$. Of course, if we have an orthogonal set it is easy to construct an orthonormal set by simply dividing each vector by its norm. Therefore, the hard work lies in finding orthogonal sets.

In this section, we will study a general procedure for constructing an orthogonal set of vectors from any given set of vectors, and use this method to define an important matrix decomposition called the _QR decomposition_. 

## The Gram--Schmidt procedure

Suppose we have a set of vectors $a_1, \dots, a_k \in \mathbb{R}^n$, which we might think of as being the columns of a $n\times k$ matrix $A$. Can we find a set of orthonormal vectors $v_1,\dots, v_k$ such that $\text{span}(a_1,\dots, a_k) = \text{span}(v_1,\dots, v_k)$? It turns out that we can use an algorithm called the _Gram--Schmidt procedure_ (or _Gram--Schmidt process_) to accomplish this. 

The algorithm procedes as follows. Start with vectors $a_1,\dots, a_k$. Then proceed as follows:

- $u_1 = a_1$, and set $v_1 = \frac{u_1}{\|u_1\|_2}$
- $u_2 = a_2 - \frac{a_2^\top u_1}{u_1^\top u_1}u_1 = a_2 - (a_2^\top v_1)v_1$, and set $v_2 = \frac{u_2}{\|u_2\|_2}$
- $u_3 = a_3 - \frac{a_3^\top u_1}{u_1^\top u_1}u_1 - \frac{a_3^\top u_2}{u_2^\top u_2}u_2 = a_3 - (a_3^\top v_1)v_1 - (a_3^\top v_2)v_2$, and set $v_3 = \frac{u_3}{\|u_3\|_2}$
- $\vdots $
- $u_k = a_k - \sum_{j=1}^{k-1} \frac{a_k^\top u_j}{u_j^\top u_j}u_j = a_k - \sum_{j=1}^{k-1}(a_k^\top v_j)v_j$, and set $v_k = \frac{u_k}{\|u_k\|_2}$ 

The vectors $u_1,\dots, u_k$ are the important ones here: they form an orthogonal set with the same span as $a_1,\dots, a_k$. The set $v_1,\dots,v_k$ are simply the normalized versions of $u_1,\dots,u_k$, which are therefore an _orthonormal_ set. 

To see why this procedure works, let's look at just the first step, and check that $u_1$ and $u_2$ are in fact orthogonal. To do this, we want to verify that $u_1^\top u_2 = 0$. We have


$$
u_1^\top u_2 = a_1^\top (a_2 - \frac{a_2^\top a_1}{a_1^\top a_1}a_1) = a_1^\top a_2 - a_1^\top a_1 \frac{a_2^\top a_1}{a_1^\top a_1} = a_1^\top a_2 - a_2^\top a_1 = a_1^\top a_2 - a_1^\top a_2 = 0
$$


In the second to last inequality, we used the fact that $x^\top y = y^\top x$ for any vectors $x,y$. The same type of calculation can be used to check that $u_i^\top u_j = 0$ for any $i\neq j$. Thus the vectors are indeed orthogonal. Moreover, we can see that $u_1,\dots,u_k$ must have the same span as $a_1,\dots, a_k$, since $u_j$ can be written as a linear combination of $a_1,\dots, a_j$ for any $j$. 

### Implementing the Gram--Schmidt procedure in Python

Let's use Python and numpy to implement the Gram--Schmidt algorithm. 

Let's start with a few helper functions. First, we'll implement a function which takes vectors $u$ and $v$ and computes $\frac{v^\top u}{u^\top u}u$. As we will see in a later section, this is the _orthogonal projection of $v$ onto $u$_, so to be consistent with that interpretation, we will call this function `project_v_onto_u`.

In [ ]:
import numpy as np

def project_v_onto_u(v,u):
    return (np.dot(v,u)/np.dot(u,u))*u

Next, let's define a function `normalize` which takes a vector $u$ and returns a unit vector in the same direction: $v = \frac{u}{\|u\|_2}$.

In [ ]:
def normalize(u):
    return u/np.linalg.norm(u)

Finally, let's define a function `gram_schmidt` which uses our helper functions to compute the orthonormal vectors $v_1,\dots, v_k$ given a set $a_1,\dots, a_k$. To do this, we need to decide how we should take the vectors $a_1,\dots, a_k$ as inputs. We will choose to assume that the input is a $n\times k$ matrix $A$, which has the vectors $a_1,\dots, a_k$ as its columns. This will be convenient later on when we compute the QR decomposition. Then we will have our function output a matrix $Q$ whose columns are $v_1,\dots, v_k$ -- again, this will be convenient for later on.

Our function will work as follows: we will have an outer for loop which loops through $i=1,\dots, k$. Then, we will have an inner for loop which loops through $j=1,\dots,i-1$ and iteratively subtracts $\frac{a_i^\top u_j}{u_j^\top u_j}u_j$ from $a_i$.

In [ ]:
def gram_schmidt(A):
    k = A.shape[1]
    u_list = [] # initialize a list to store the u vectors
    u_list.append(A[:, 0]) # u1 = a1
    for i in range(1,k):
        ui = A[:, i] # start with ui = ai
        for j in range(i):
            ui = ui - project_v_onto_u(ui, u_list[j]) # subtract out all the components (ai^T uj)/(uj^T uj)*uj
        u_list.append(ui) # add ui to the list of u vectors
    v_list = [normalize(u) for u in u_list] # normalize all the u vectors
    Q = np.stack(v_list, axis=1) # store the orthonormal vectors into a matrix Q
    return Q

Let's test our function on a random matrix $A$, and make sure that the matrix $Q$ that we get back does in fact have orthonormal columns -- that is, $Q$ should be an _orthogonal matrix_.

In [ ]:
k = 5
n = 10

A = np.random.normal(size = (n,k))
Q = gram_schmidt(A)

Recall that we can check that $Q$ is an orthogonal matrix by checking if $Q^\top Q = I$. Let's see that this is in fact true. Again, we round to 8 decimals to make the matrix easier to read.

In [ ]:
np.round(np.dot(Q.T, Q), 8)

Indeed, we see that $Q^\top Q$ is in fact the identity matrix.

**Remark:** At this point, it is important to point out that the orthogonal matrix $Q$ whose columns have the same span as $A$ is not exactly unique. Indeed, it's easy to see that if we multiply any of the columns of $Q$ by $-1$, we will have an orthogonal matrix with columns spanning the column space of $A$. 

## From Gram-Schmidt to QR

Now that we've seen how to take the columns of an arbitrary matrix $A$ and come up with an orthonormal set spanning the column space of $A$, we are in a position to introduce one of the most important matrix decompositions in linear algebra: the _QR decomposition_. In the QR decomposition, we write any matrix $A$ as a product $A = QR$ where $Q$ is an orthogonal matrix, and $R$ is an upper triangular matrix. 

Let's start with the orthonormal vectors $v_1,\dots, v_k$ that we obtain from the Gram-Schmidt procedure. Importantly, we can write the columns of $A$ as a linear combination of the vectors $v_1,\dots, v_k$. To see how this works, note that from the Gram--Schmidt procedure we have for any $j=1,\dots,k$,


$$
u_j = a_j - \sum_{i=1}^{j-1}(a_j^\top v_i)v_i \iff a_j = u_j + \sum_{i=1}^{j-1}(a_j^\top v_i)v_i = \|u_j\|_2v_j +\sum_{i=1}^{j-1}(a_j^\top v_i)v_i
$$
 

Where for the last equality we used the fact that $v_j = \frac{u_j}{\|u_j\|_2}$. Now notice that


$$
\|u_j\|_2^2 = u_j^\top u_j = u_j^\top \left(a_j - \sum_{i=1}^{j-1}(a_j^\top v_i)v_i\right) = u_j^\top a_j - \sum_{i=1}^{j-1}(a_j^\top v_i)\underbrace{(u_j^\top v_i)}_{=0}= u_j^\top a_j = \|u_j\|_2(v_j^\top a_j) \implies \|u_j\|_2 = (v_j^\top a_j)
$$


Hence we get the following expression for $a_j$:


$$
a_j = \|u_j\|_2v_j +\sum_{i=1}^{j-1}(a_j^\top v_i)v_i = (a_j^\top v_j)v_j + \sum_{i=1}^{j-1}(a_j^\top v_i)v_i = \sum_{i=1}^j (a_j^\top v_i)v_i \hspace{20mm} (\star)
$$


Therefore $a_j$ can be written as a linear combination of $c_1 v_1 + \cdots + c_jv_j$ where $c_i = (a_j^\top v_i)$. 

Now let's collect the coefficients $(a_j^\top v_j)$ into the following matrix:


$$
R = \begin{pmatrix}a_1^\top v_1 & a_2^\top v_1 & a_3^\top v_1&\cdots  \\
									0 & a_2^\top v_2 &a_3^\top v_2& \cdots \\
									0 & 0 & a_3^\top v_3 & \cdots \\
									\vdots & \vdots &\vdots &\ddots \\
									
\end{pmatrix}
$$


That is, the $k\times k$ matrix $R$ whose $(i,j)$th entry is $a_j^\top v_i$ if $i\leq j$, and $0$ otherwise (matrices of this form -- with zeros below the diagonal -- are called _upper triangular_). Let's again store the vectors $v_1,\dots, v_k$ as the columns of a $n\times k$ 


$$
Q = \begin{pmatrix} | & | &  &| \\ v_1 & v_2 & \cdots & v_k \\ | & | & & |\end{pmatrix}
$$


Using this notation, we can write the relationship $(\star)$ as $a_j = Qr_j$, where $r_j$ is the $j$th column of the matrix $R$. In particular then, if we stack all these columns together, we get that 


$$
A = QR = \begin{pmatrix} | & | & &| \\ v_1 & v_2 & \cdots & v_k \\ | & | &  & |\end{pmatrix}\begin{pmatrix}a_1^\top v_1 & a_2^\top v_1 & a_3^\top v_1&\cdots  \\
									0 & a_2^\top v_2 &a_3^\top v_2& \cdots \\
									0 & 0 & a_3^\top v_3 & \cdots \\
									\vdots & \vdots &\vdots &\ddots \\
									
\end{pmatrix}
$$


This expression -- writing $A$ as a product of an orthogonal matrix $Q$ and an upper triangular matrix of coefficients $R$ -- is called the _$QR$ decomposition of $A$_. In words, this decomposition expresses the columns of $A$ in terms of an orthogonal basis $Q$, which we obtain through Gram--Schmidt.

### Computing the QR decomposition in Python

Let's implement the QR decomposition in Python. Since we've already implemented Gram-Schmidt above, we can use that function to obtain the matrix $Q$. Thus, all we have left to do is find the upper triangular matrix $R$. We could go through and compute all the entries of $R$ manually, however, if we notice that $Q^\top Q= I$, we observe that 


$$
A = QR \iff Q^\top A = Q^\top QR = R
$$


Thus we can compute $R$ immediately by calculating $Q^\top A$. Let's combine all these steps into a single function which computes $Q$ and $R$ for any given matrix $A$.

In [ ]:
def qr_decomposition(A):
    Q = gram_schmidt(A) #use Gram-Schmidt to compute Q
    R = np.dot(Q.T, A) #find R = Q^TA
    return Q, R

Let's test this again on a random matrix $A$.

In [ ]:
k = 5
n = 10

A = np.random.normal(size = (n,k))
Q, R = qr_decomposition(A)

Now let's check that $R$ is indeed upper triangular.

In [ ]:
R.round(8)

Let's also check that $A = QR$.

In [ ]:
np.allclose(A, np.dot(Q,R))

Indeed, it does. For this section, we simply focus on the mechanics of the QR decomposition. In the following sections of this chapter, we will see that this is an extremely useful tool.